# 1.3　怎样寻找最好的算法

## 例题1.3　总和最大区间问题　（难度系数3颗星)
给定一个实数序列，设计一个最有效的算法，找到一个总和最大的区间。

比如在下面的序列中：

1.5, −12.3, 3.2, −5.5, 23.2, 3.2, −1.4, −12.2, 34.2, 5.4, −7.8, 1.1, −4.9

总和最大的区间是从第5个数（23.2）到第10个数（5.4）。

这个问题我还见过另一种表述，即寻找一只股票最长的有效增长期。研究股票投资的人都想了解一只股票最长的有效增长期是哪一个时间段。

以上数组可以认为是一只股票每天的涨跌幅度。

### 方法1，做一次三重循环，其实就是中学里学的排列组合的方法。

我们假设这个序列有K个数，依次是a1,a1,a2,…,aK。假定区间起始的数字序号为p，结束的数字序号为q，这些数字的总和为S(p,q)，则S(p,q)=ap+ap+1+…+aq。

p可以从1一直到K，q可以从p一直到K，这是两重循环了，因此区间一头一尾的组合有O(K2)种。在每一种组合中，计算S(p,q)平均要做K/4次加法，这是又一重循环。因此这种算法的复杂度是O(K3)。

In [9]:
from typing import List,Tuple
class Solution1:
    def maxProfitRange(self, variation: List[int]) -> Tuple[int]:
        total = float("-inf")
        k = len(variation)
        res = (0,k-1)
        for p in range(k): # 第一重循环
            for q in range(p+1,k): # 第二重循环
                cur = sum(variation[p:q+1]) #第三重循环，cur即S(p,q)
                if cur > total:
                    total = cur
                    res = (p,q)
        return res        

In [10]:
ups_downs = [1.5, -12.3, 3.2, -5.5, 23.2, 3.2, -1.4, -12.2, 34.2, 5.4, -7.8, 1.1, -4.9]
Solution1().maxProfitRange(ups_downs)

(4, 9)

以0为下标起始点则区间是（4，9），以1为起始点则是（5，10）

算法的复杂度是O(K**3)

### 方法2，做两重循环。

方法1效率不高的原因是做了太多的无用功，比如当我们把区间的起点定在了位置p之后，如果已经计算了从p到q之间的数字的总和S(p,q)，下次再计算从p到q+1之间的数字的总和S(p,q+1)时，只需要在原来的基础上再做一次加法，而不需要再来一次循环。当然有人可能会担心，这样是否需要占用额外的存储空间来保留所有的中间结果S(p,q)。其实这种担心是不必要的，因为我们只需要记录这样三个中间值。

第一个值是从p开始到当前位置q为止的总和S(p,q)，因为我们接下来计算S(p,q+1)时要用到它。

第二个值则是从p开始到当前位置q为止所有总和中最大的那个值，我们假定为Max。有了这个值之后，如果S(p,q+1)≤Max，则Max维持不变；如果S(p,q+1)>Max，则要更新Max，当然，我们也要记录下来Max是在区间[p,q+1]取得的。

因此，第三个要记录的值就是区间结束的位置，我们不妨以r来表示。如果Max的值更新了，相应的区间结束位置也要更新为q+1。

In [16]:
class Solution2:
    def maxProfitRange(self, variation: List[int]) -> Tuple[int]:
        Spq = 0
        k = len(variation)
        res = (0,k-1)
        Maxv = 0
        for p in range(k): # 第一重循环
            Spq = variation[p]
            for q in range(p+1,k): # 第二重循环
                Spq += variation[q]
                if Spq > Maxv:
                    Maxv = Spq
                    res = (p,q)
        return res

In [17]:
ups_downs = [1.5, -12.3, 3.2, -5.5, 23.2, 3.2, -1.4, -12.2, 34.2, 5.4, -7.8, 1.1, -4.9]
Solution2().maxProfitRange(ups_downs)

(4, 9)

### 方法3，利用分治（Divide-and-Conquer）算法。
首先，将序列一分为二，分成从1到K/2[5]，以及从K/2+1到K两个子序列。

然后，我们对这两个子序列分别求它们的总和最大区间。接下来有两种情况。

1．前后两个子序列的总和最大区间中间没有间隔，也就是说，前一个子序列的总和最大区间是[p,K/2]，后一个总和最大区间恰好是[K/2+1,q]。如果两个区间各自的和均为正整数，这时，整个序列总和最大区间就是[p, q]；否则，就选取两个子序列的总和最大区间中大的一个。

2．前后两个子序列的总和最大区间中间有间隔，我们假定这两个子序列的总和最大区间分别是[p1,q1]和[p2,q2]。这时，整个序列的总和最大区间是下面三者中最大的那一个：

（1）[p1,q1]；

（2）[p2,q2]；

（3）[p1,q2]。

In [37]:
class Solution3:
    def maxProfitRange(self, variation: List[int]) -> Tuple[int]:
        k = len(variation)
        if k == 1:   # base case
            return (0,1)
        p1, q1 = self.maxProfitRange(variation[:k//2]) # devide
        p2, q2 = self.maxProfitRange(variation[k//2:]) # devide
        p2 += k//2 +1
        q2 += k//2 +1
        left = sum(variation[p1:q1])
        right = sum(variation[p2:q2])
        if q1 == p2:
            if left > 0 and right > 0:
                return (p1, q2)
            elif left > right:
                return (p1,q1)
            return (p2, q2)
        total = sum(variation[p1:q2])
        if total > left and total > right:
            return (p1, q2)
        elif left > total and left > right:
            return (p1, q1)
        return (p2,q2)

In [36]:
ups_downs = [1.5, -12.3, 3.2, -5.5, 23.2, 3.2, -1.4, -12.2, 34.2, 5.4, -7.8, 1.1, -4.9]
Solution3().maxProfitRange(ups_downs)

(4, 9)

### 方法4，正、反两遍扫描的方法
- 在方法2基础上的改进
